In [3]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.llms import Ollama
from langchain.schema import HumanMessage, SystemMessage

In [5]:
from langchain.utilities import SQLDatabase

connection_string = 'sqlite:///data/sample.db'
db = SQLDatabase.from_uri(connection_string)
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['maintenance', 'universityconfigurations']


In [6]:
from langchain_ollama.chat_models import ChatOllama
llm = ChatOllama(model="project")
llm.invoke("Who are you ?")

AIMessage(content='Hello! I am your intelligent chat assistant for the Technical University of Ilmenau. I am here to assist you with your 5G technology research.\n\nHow can I help you today?', response_metadata={'model': 'project', 'created_at': '2024-08-19T18:43:03.119908Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4049856208, 'load_duration': 45374708, 'prompt_eval_count': 493, 'prompt_eval_duration': 2646992000, 'eval_count': 40, 'eval_duration': 1349137000}, id='run-d446c749-ed8c-4acf-bed4-8a1b7809227d-0', usage_metadata={'input_tokens': 493, 'output_tokens': 40, 'total_tokens': 533})

In [7]:
context = db.get_context()
print(list(context))
print(context["table_info"])

['table_info', 'table_names']

CREATE TABLE maintenance (
	"Maintenance_ID" INTEGER, 
	"Base_Station" TEXT NOT NULL, 
	"Maintenance_Date" TEXT NOT NULL, 
	"Technician_Name" TEXT NOT NULL, 
	"Description" TEXT, 
	"Duration_hours" REAL NOT NULL, 
	"Cost" REAL NOT NULL, 
	PRIMARY KEY ("Maintenance_ID"), 
	FOREIGN KEY("Base_Station") REFERENCES universityconfigurations ("Base_Station")
)

/*
3 rows from maintenance table:
Maintenance_ID	Base_Station	Maintenance_Date	Technician_Name	Description	Duration_hours	Cost
1	BS-001	2023-06-15	John Doe	Routine check-up	2.0	150.0
2	BS-002	2023-07-10	Jane Smith	Antenna alignment	3.0	200.0
3	BS-003	2023-07-20	Emily Johnson	Software update	1.5	100.0
*/


CREATE TABLE universityconfigurations (
	"ID" INTEGER NOT NULL, 
	"Base_Station" TEXT NOT NULL, 
	"Location" TEXT NOT NULL, 
	"Bandwidth" REAL NOT NULL, 
	"Transmission_Power" REAL NOT NULL, 
	"Frequency_Band" REAL NOT NULL, 
	"Coverage_Area" REAL NOT NULL, 
	"Maximum_Capacity" INTEGER NOT NULL, 
	"Opera

In [8]:
from langchain.chains import create_sql_query_chain
custom_template='''You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the 
input question.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
If mot mentioned do not use {top_k} rows. Output only the SQL Query without any output or result of the query.
Make sure you are replacing ```sql word to blank space. If the answer could not be found, give a single line meaningful error

Context:
    CREATE TABLE maintenance (
	"Maintenance_ID" INTEGER, 
	"Base_Station" TEXT NOT NULL, 
	"Maintenance_Date" TEXT NOT NULL, 
	"Technician_Name" TEXT NOT NULL, 
	"Description" TEXT, 
	"Duration_hours" REAL NOT NULL, 
	"Cost" REAL NOT NULL, 
	PRIMARY KEY ("Maintenance_ID"), 
	FOREIGN KEY("Base_Station") REFERENCES universityconfigurations ("Base_Station")
)

	CREATE TABLE "universityconfigurations" (
    ID INTEGER NOT NULL,
    Base_Station TEXT NOT NULL,
    Location TEXT NOT NULL,
    Bandwidth REAL NOT NULL,
    Transmission_Power REAL NOT NULL,
    Frequency_Band REAL NOT NULL,
    Coverage_Area REAL NOT NULL,
    Maximum_Capacity INTEGER NOT NULL,
    Operational_Status TEXT NOT NULL,
    Deployment_Date TEXT NOT NULL,
    PRIMARY KEY (Base_Station)
)

For Base stations id's always use BS-00 in the prefix. For example, if base station 4 is asked use 'BS-004'.
    
Only use the following tables:
    {table_info}.


Question: {input}'''
example_prompt = PromptTemplate(
input_variables=["input", "top_k","table_info"], template=custom_template)
write_query = create_sql_query_chain(llm, db,prompt=example_prompt)
# write_query = create_sql_query_chain(llm, db)
response = write_query.invoke({"question": "Count the number of base stations?"})
response

"```sql\nSELECT COUNT(*) AS num_base_stations\nFROM universityconfigurations\nWHERE Base_Station = 'BS-001';\n```"

In [ ]:
response = write_query.invoke({"question": "What is the station id?"})
response